In [29]:
import pygame
import numpy as np 
import pymunk
import pymunk.pygame_util
from pymunk import Vec2d
from itertools import combinations

class Tiles:
    def __init__(self, 
                 num, 
                 seed, 
                 mass, 
                 length, 
                 height, 
                 tile_friction, 
                 table_static_friction, 
                 table_kinetic_friction, 
                 hand_velocity
        ):
        """
        take g=1
        """
        self.num=num
        self.seed=seed
        self.mass=mass
        self.length=length
        self.height=height
        self.tile_friction=tile_friction
        self.table_static_friction=table_static_friction
        self.table_kinetic_friction=table_kinetic_friction
        self.hand_velocity=hand_velocity
        
        self.frictional_torque_coeff = (mass/(12*height*length))*(
            2*height*length*np.sqrt(height**2 + length**2) + (
                height**3 * np.log((length + np.sqrt(height**2 + length**2))/height)
            ) + (
                length**3 * np.log((height + np.sqrt(length**2 + height**2))/length)
            )
        )
        
        self.running = True
        self.drawing = True
        self.w, self.h = 30*max(length, height), 30*max(length, height)
        self.screen = pygame.display.set_mode((self.w, self.h))
        self.clock = pygame.time.Clock()
        
        self.space = pymunk.Space()
        self.space.sleep_time_threshold = 0.3
        
        positions = self.intial_positions()
        
        for position in positions:
            points = [(-length/2, -height/2), (-length/2, height/2), (length/2, height/2), (length/2, height/2)]
            mass = self.mass
            moment = pymunk.moment_for_poly(mass, points, (0, 0))
            body = pymunk.Body(mass, moment)
            body.position = Vec2d(*position) 
            body.angle = np.random.rand()*2*np.pi
            shape = pymunk.Poly(body, points)
            shape.friction = self.tile_friction
            self.space.add(body, shape)
        
        pymunk.pygame_util.positive_y_is_up = True
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        
    def initial_positions(self):
        """
        assign initial positions and orientations to tiles
        """
        box_width = np.sqrt(self.height**2 + self.length**2)
        
        radius = int(1+np.sqrt(1.5*self.num / np.pi))
        
        box_sites = list(combinations(np.arange(-radius, radius), 2))
        
        sorted_positions = sorted(box_sites, key = lambda x: x[0]**2 + x[1]**2)
        
        scaled_sorted_positions = sorted_positions * box_width
        
        return scaled_sorted_positions[:self.num]
    
    def bring_tiles_together(self):
        """
        assign forces on all tiles to collect them together
        """
        for shape in self.space.shapes:
            position = shape.body.position
            
            shape.body.apply_force_at_local_point(-position / np.sqrt(position.get_length_sqrd()))
        
        return None
    
    def collect_handful(self, hand_radius, angle_of_approach):
        """
        Find a handful of tiles and return their indices
        """
        approach_vector = (np.cos(angle_of_approach), np.sin(angle_of_approach))
        
        projections = []
        
        for idx, shape in enumerate(self.space.shapes):
            
            projection = (
                shape.body.position[0]*approach_vector[0]+shape.body.position[1]*approach_vector[1]
            )
            
            projections.append((shape.body.position, projection))
        
        projections = sorted(projections, key = lambda x: x[-1])
        
        hand_position = projections[-1][0]
        
        opposite_position = projections[0][0]
        
        """
        Now collect all tiles in the radius
        """
        
        tiles = []
        
        for idx, shape in enumerate(self.space.shapes):
            if shape.body.position.get_distance(hand_position)<hand_radius:
                tiles.append(idx)
        
        return tiles, hand_position, opposite_position
    
    def assign_velocities(self, velocity, tile_indices):
        """
        
        """
        for idx in tile_indices:
            self.space.shapes[idx].body.velocity=velocity
        
    def assign_friction_forces(self):
        """
        Here we assign friction force updates 
        
        This includes the linear friction, as well as the rotating velocity
        
        This is also a function of the static and kinetic coefs of friction
        """
        self.table_static_friction
        
        for shape in self.space.shapes:
            """
            if the force is nonzero, add an opposing force
            
            if velocity is zero, use static friction
            else use kinetic friction
            
            """
            
            if shape.body.velocity == shape.body.angular_velocity == 0:
                
                current_force = np.sqrt(shape.body.force.get_length_sqrd())
                
                current_torque = np.abs(shape.body.torque)
                
                
            
                if self.table_static_friction*self.mass >= current_force:
                    shape.body.apply_force_at_local_point(
                        -shape.body.force
                    )
                
                else:
                    shape.body.apply_force_at_local_point(
                        -self.table_static_friction*self.mass*shape.body.force/current_force
                    )
                
                static_friction_torque=self.frictional_torque_coeff*self.table_static_friction*self.mass
                
                if  static_friction_torque> current_torque:
                    shape.body.torque=0
                
                else:
                    shape.body.torque=shape.body.torque*(1-static_friction_torque/current_torque)
            
            else:
                # use kinetic_friction
                
                     
            
            shape.body.force = current_force*(1-self.table_static_friction*self.mass/current_force)
            apply_force_at_local_point(-position)
            
            position = shape.body.position
            
            shape.body.apply_force_at_local_point(-position)
        
        return None
    
    def all_distances(self):
        """
        return the n by n matrix of all positions
        """
        dists = np.zeros((self.num, self.num))
        
        for i, shape in enumerate(self.space.shapes):
            for j in range(i):
                dists[i, j]=second_shape.body.position.get_dist_sqrd(shape.body.position)
                dists[j, i]=dists[i, j]
        return None
    
    def spatial_correlation(self):
        """
        compute the current spatial correlation
        """
        return None
        

Integrate[Sqrt[x^2 + y^2], {x, -w/2, w/2}, {y, -h/2, h/2}]

In [20]:
list(combinations([1,2, 3], 2))

[(1, 2), (1, 3), (2, 3)]

In [28]:
Vec2d(1, 2)

TypeError: unsupported operand type(s) for ** or pow(): 'Vec2d' and 'int'

Make a board covered in tiles

initialize it by evolving under a central potential to cluster them together

define a function to push tiles around 

define a function to squeeze them together again 

What is the initial to final distance correlation as a function of the number of reps

In [21]:
import pygame

import pymunk
import pymunk.pygame_util
from pymunk import Vec2d


class PyramidDemo:
    def __init__(self):
        self.running = True
        self.drawing = True
        self.w, self.h = 600, 600
        self.screen = pygame.display.set_mode((self.w, self.h))
        self.clock = pygame.time.Clock()

        ### Init pymunk and create space
        self.space = pymunk.Space()
        #self.space.gravity = (0.0, 0.0)
        self.space.sleep_time_threshold = 0.3
        ### ground
        #shape = pymunk.Segment(self.space.static_body, (5, 100), (595, 100), 1.0)
        #shape.friction = 1.0
        #self.space.add(shape)

        ### pyramid
        x = Vec2d(-270, 7.5) + (300, 100)
        y = Vec2d(0, 0)
        deltaX = Vec2d(0.5625, 1.1) * 20
        deltaY = Vec2d(1.125, 0.0) * 20

        for i in range(25):
            y = Vec2d(*x)
            for j in range(i, 25):
                size = 10
                points = [(-size, -size), (-size, size), (size, size), (size, -size)]
                mass = 1.0
                moment = pymunk.moment_for_poly(mass, points, (0, 0))
                body = pymunk.Body(mass, moment)
                body.position = y 
                shape = pymunk.Poly(body, points)
                shape.friction = 1
                self.space.add(body, shape)

                y += Vec2d(1.125+np.random.randn(), 0.0) * 20 

            x += deltaX
        

        ### draw options for drawing
        pymunk.pygame_util.positive_y_is_up = True
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)

    def run(self):
        while self.running:
            self.loop()

    def loop(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                self.running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_p:
                pygame.image.save(self.screen, "box2d_pyramid.png")
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_d:
                self.drawing = not self.drawing

        fps = 30.0
        dt = 1.0 / fps / 5
        self.space.step(dt)
        
        #self.space.shapes[1].body.apply_force_at_local_point((0, 400))
        
        #self.space.shapes[100].body.apply_force_at_local_point((0, 400))
        
        self.space.shapes[100].body.torque=700
        
        #self.space.shapes[100].body.velocity=Vec2d(0.0, 100.0)
        
        if self.drawing:
            self.draw()

        ### Tick clock and update fps in title
        self.clock.tick(fps)
        pygame.display.set_caption("fps: " + str(self.clock.get_fps()))

    def draw(self):
        ### Clear the screen
        self.screen.fill(pygame.Color("white"))

        ### Draw space
        self.space.debug_draw(self.draw_options)

        ### All done, lets flip the display
        pygame.display.flip()


def main():
    demo = PyramidDemo()
    demo.run()


if __name__ == "__main__":
    main()

In [22]:
demo = PyramidDemo()

In [4]:
vars(demo.space)

{'threaded': False,
 '_space': <cdata 'struct cpSpace *' 0x000001C364B94890>,
 '_handlers': {},
 '_post_step_callbacks': {},
 '_removed_shapes': {},
 '_shapes': {326: <pymunk.shapes.Poly at 0x1c3659b2bc8>,
  327: <pymunk.shapes.Poly at 0x1c365aa24c8>,
  328: <pymunk.shapes.Poly at 0x1c365aa2c88>,
  329: <pymunk.shapes.Poly at 0x1c365aa4048>,
  330: <pymunk.shapes.Poly at 0x1c365aa43c8>,
  331: <pymunk.shapes.Poly at 0x1c365aa6448>,
  332: <pymunk.shapes.Poly at 0x1c365aa6c48>,
  333: <pymunk.shapes.Poly at 0x1c365aa8308>,
  334: <pymunk.shapes.Poly at 0x1c365aa8888>,
  335: <pymunk.shapes.Poly at 0x1c365aa8d88>,
  336: <pymunk.shapes.Poly at 0x1c365aaa748>,
  337: <pymunk.shapes.Poly at 0x1c365aaaec8>,
  338: <pymunk.shapes.Poly at 0x1c365aabbc8>,
  339: <pymunk.shapes.Poly at 0x1c365aabd08>,
  340: <pymunk.shapes.Poly at 0x1c365aa0808>,
  341: <pymunk.shapes.Poly at 0x1c365aa02c8>,
  342: <pymunk.shapes.Poly at 0x1c365aac988>,
  343: <pymunk.shapes.Poly at 0x1c365aac448>,
  344: <pymu

In [5]:
demo.space.shapes

In [6]:
vars(demo.space.shapes[1])

{'_body': Body(1.0, 66.66666666666667, Body.DYNAMIC),
 '_shape': <cdata 'struct cpShape *' 0x000001C364D6DF10>,
 '_space': <pymunk.space.Space at 0x1c36421b778>}

In [7]:
vars(demo.space.shapes[1])

{'_body': Body(1.0, 66.66666666666667, Body.DYNAMIC),
 '_shape': <cdata 'struct cpShape *' 0x000001C364D6DF10>,
 '_space': <pymunk.space.Space at 0x1c36421b778>}

In [8]:
vars(demo.space.shapes[1].body)

{'_body': <cdata 'struct cpBody *' 0x000001C364D68430>,
 '_position_func': None,
 '_velocity_func': None,
 '_position_func_base': None,
 '_velocity_func_base': None,
 '_space': <pymunk.space.Space at 0x1c36421b778>,
 '_constraints': <_weakrefset.WeakSet at 0x1c3659b2348>,
 '_shapes': <_weakrefset.WeakSet at 0x1c3659b2588>}

In [9]:
demo.space.shapes[1].body.angular_velocity

0.0

In [10]:
vars(demo.space.shapes[1].body)

{'_body': <cdata 'struct cpBody *' 0x000001C364D68430>,
 '_position_func': None,
 '_velocity_func': None,
 '_position_func_base': None,
 '_velocity_func_base': None,
 '_space': <pymunk.space.Space at 0x1c36421b778>,
 '_constraints': <_weakrefset.WeakSet at 0x1c3659b2348>,
 '_shapes': <_weakrefset.WeakSet at 0x1c3659b2588>}

In [11]:
demo.space.shapes[1].body.angular_velocity

0.0

In [12]:
demo.space.shapes[1].body.torque

0.0

In [24]:
demo.space.shapes[1].body.angle

0.0

In [30]:
demo.space.shapes[1].body.force

Vec2d(0.0, 0.0)

In [14]:
Vec2d(0.0, 0.0)

Vec2d(0.0, 0.0)

In [15]:
vars(demo.space.space)

AttributeError: 'Space' object has no attribute 'space'

In [26]:
demo.space.shapes[1].body.position[0]

65.59019056154725

In [12]:
num_points = 1000

In [55]:
np.random.seed(1)

In [56]:
gaussian = (1 + np.sqrt(2)*np.random.randn(num_points, num_points)/4)/2

In [57]:
gaussian = gaussian+ np.transpose(gaussian)

In [58]:
gaussian_diag = np.diag(1+np.random.randn(num_points)/4)

In [59]:
gaussian_diag

array([[0.76215651, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.38178352, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.0545224 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.21592385, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.84846624,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.91327397]])

In [60]:
np.mean(np.diag(gaussian_diag))

1.0023759732186257

In [61]:
np.std(np.diag(gaussian_diag))

0.245714997470168